### Author: Baoling Zhou
### This notebook script is created for reconstructering NYC Yellow Taxi Data into Spatial-temporal format in terms of zipcode area and monthly total passenger counts.
### Everytime before running the script, please restart the kernel and define the year and month you want to analyze. It usually takes 40 minutes to run a monthly data in size of 2 G.

In [64]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import geopandas as gpd
from shapely.geometry import Point
import fiona
import math
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt', 'test']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


## Define the year and month to process

In [65]:
Year = {9:'09',10:'10',11:'11',12:'12',13:'13',14:'14',15:'15',16:'16',17:'17'}
Month = {1:'01',2:'02',3:'03',4:'04',5:'05',6:'06',7:'07',8:'08',9:'09',10:'10',11:'11',12:'12'}

year = Year[16] #define the year
month = Month[4] #define the month
print (year+month)

1604


## Download the taxi data

In [66]:
#Download the data
df = pd.read_csv('https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_20' + year + '-' + month + '.csv')

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11934338 entries, 0 to 11934337
Data columns (total 19 columns):
VendorID                 int64
tpep_pickup_datetime     object
tpep_dropoff_datetime    object
passenger_count          int64
trip_distance            float64
pickup_longitude         float64
pickup_latitude          float64
RatecodeID               int64
store_and_fwd_flag       object
dropoff_longitude        float64
dropoff_latitude         float64
payment_type             int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
dtypes: float64(12), int64(4), object(3)
memory usage: 1.7+ GB


In [68]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RatecodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')

In [69]:
## remain the useful columns, reduce the memory stored in the kernel
df = df[['passenger_count','pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11934338 entries, 0 to 11934337
Data columns (total 5 columns):
passenger_count      int64
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
dtypes: float64(4), int64(1)
memory usage: 455.3 MB


## Step 1: Clean component data

## Step 2: Concate 3 components into a month

In [70]:
df.head()

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,1,-73.976883,40.758495,-73.977669,40.753902
1,2,-73.985207,40.757294,-73.989288,40.732658
2,2,-73.979202,40.758869,-73.990677,40.751320
3,5,-73.984856,40.767723,-73.990829,40.751186
4,2,-73.863739,40.769470,-73.976814,40.775284


In [71]:
df.shape

(11934338, 5)

### Binning by Zipcode for data with coordinates info

In [72]:
#read zipcode shape file, define a constant crs
zipcode = gpd.read_file('ZIP_CODE/ZIP_CODE_040114.shp')
zipcode = zipcode[['ZIPCODE','geometry']]
zipcode = zipcode.to_crs(epsg=4326)
crs = {'init' :'epsg:4326'}

In [73]:
# pick = [Point(xy) for xy in zip(df.pickup_longitude, df.pickup_latitude)]
# drop = [Point(xy) for xy in zip(df.dropoff_longitude, df.dropoff_latitude)]

In [74]:
#create the coordinates columns fulfilling the geodataframe format
pick = []
drop = []
for i in range(math.floor(len(df)/1000000)+1):
    start = 1000000 * i
    if i < math.floor(len(df)/1000000):
        end = 1000000 * (i+1)
    else:
        end = len(df)
    test = df.iloc[start:end,:]
    pick_ = [Point(xy) for xy in zip(test.pickup_longitude, test.pickup_latitude)]
    drop_ = [Point(xy) for xy in zip(test.dropoff_longitude, test.dropoff_latitude)]
    pick = pick + pick_
    drop = drop + drop_
    print(str(i) + '/' + str(math.floor(len(df)/1000000)))

0/11
1/11
2/11
3/11
4/11
5/11
6/11
7/11
8/11
9/11
10/11
11/11


In [75]:
#double check the length of the geometry list
len(pick)

11934338

In [76]:
#convert the dataframe into geodataframe
pick_gdf = gpd.GeoDataFrame(df[['passenger_count']], geometry=pick, crs=crs)
drop_gdf = gpd.GeoDataFrame(df[['passenger_count']], geometry=drop, crs=crs)

In [77]:
#clean the list, reduce memory stored in the kernel
pick = []
drop = []

In [78]:
#read the template as an empty table to prepare binning
tmp1 = pd.read_csv('Yellow_taxi_data/binning_tmp.csv')
tmp2 = pd.read_csv('Yellow_taxi_data/binning_tmp.csv')
del tmp1['Unnamed: 0']
del tmp2['Unnamed: 0']

In [79]:
# binning the record to identify the zipcode for each pick-up coordinates
for i in range(math.floor(len(pick_gdf)/1000000)+1):
    start = 1000000 * i
    if i < math.floor(len(pick_gdf)/1000000):
        end = 1000000 * (i+1)
    else:
        end = len(pick_gdf)
    test = pick_gdf.iloc[start:end,:]
    test = gpd.sjoin(test, zipcode, how='right', op='within').reset_index()
    tmp1 = pd.concat([tmp1,test])
    print(str(i) + '/' + str(math.floor(len(pick_gdf)/1000000)))

0/11
1/11
2/11
3/11
4/11
5/11
6/11
7/11
8/11
9/11
10/11
11/11


In [80]:
# binning the record to identify the zipcode for each drop-off coordinates
for i in range(math.floor(len(drop_gdf)/1000000)+1):
    start = 1000000 * i
    if i < math.floor(len(drop_gdf)/1000000):
        end = 1000000 * (i+1)
    else:
        end = len(drop_gdf)
    test = drop_gdf.iloc[start:end,:]
    test = gpd.sjoin(test, zipcode, how='right', op='within').reset_index()
    tmp2 = pd.concat([tmp2,test])
    print(str(i) + '/' + str(math.floor(len(drop_gdf)/1000000)))

0/11
1/11
2/11
3/11
4/11
5/11
6/11
7/11
8/11
9/11
10/11
11/11


In [81]:
#delete the first row which is the beginning test record
pick_gdf = tmp1.iloc[1:,:]
drop_gdf = tmp2.iloc[1:,:]

In [82]:
#clean the dataframe, reduce memory stored in the kernel
tmp1=[]
tmp2=[]

In [83]:
# pick_gdf = gpd.sjoin(pick_gdf, zipcode, how='right', op='within').reset_index()
# drop_gdf = gpd.sjoin(drop_gdf, zipcode, how='right', op='within').reset_index()

In [84]:
#delete the irrelevant columns
del pick_gdf['index_right']
del pick_gdf['index_left']
del pick_gdf['geometry']

del drop_gdf['index_right']
del drop_gdf['index_left']
del drop_gdf['geometry']

In [85]:
#fill the nan as 0
pick_gdf = pick_gdf.fillna(0)
drop_gdf = drop_gdf.fillna(0)

In [86]:
# Group records by Zipcode
taxi_pick = pick_gdf.groupby(['ZIPCODE']).passenger_count.sum().reset_index().fillna(0)
taxi_drop = drop_gdf.groupby(['ZIPCODE']).passenger_count.sum().reset_index().fillna(0)

## Step 3: save monthly data into a column

In [87]:
# taxi_pick.columns = ['LocationID',1612]
# taxi_drop.columns = ['LocationID',1612]

In [88]:
#rename the columns based on month
taxi_pick.columns = ['ZIPCODE',int(year+month)]
taxi_drop.columns = ['ZIPCODE',int(year+month)]

In [89]:
taxi_drop.head()

,ZIPCODE,1604
0,00083,242666.0
1,10001,1125706.0
2,10002,345121.0
3,10003,840816.0
4,10004,105901.0


In [90]:
pick = taxi_pick
drop = taxi_drop

In [91]:
pick.head()

,ZIPCODE,1604
0,00083,273832.0
1,10001,1256293.0
2,10002,381489.0
3,10003,1002317.0
4,10004,95132.0


In [92]:
drop.head()

,ZIPCODE,1604
0,00083,242666.0
1,10001,1125706.0
2,10002,345121.0
3,10003,840816.0
4,10004,105901.0


In [93]:
pick.shape

(248, 2)

In [94]:
drop.shape

(248, 2)

## Saving the monthly data

In [96]:
#the name of the file can be redefine
#a sample is "Yellow_taxi_data/2016/Yellow16_drop_01.csv"
pick.to_csv('Yellow_taxi_data/20'+ year +'/Yellow' + year + '_pick_'+ month + '.csv')
drop.to_csv('Yellow_taxi_data/20'+ year +'/Yellow' + year + '_drop_'+ month + '.csv')

## Merge monthly data

In [98]:
# #merge the current month data with the previous cumulative monthly data
# #a sample is "Yellow_taxi_data/2016/Yellow16_drop_monthly.csv"
# df_pick = pd.read_csv('Yellow_taxi_data/20'+ year +'/Yellow' + year + '_pick_monthly1.csv')
# df_drop = pd.read_csv('Yellow_taxi_data/20'+ year +'/Yellow' + year + '_drop_monthly1.csv')
# del df_pick['Unnamed: 0']
# del df_drop['Unnamed: 0']

# pick16 = pd.read_csv('Yellow_taxi_data/20'+ year +'/Yellow'+ year + '_pick_'+ month + '.csv')
# drop16 = pd.read_csv('Yellow_taxi_data/20'+ year +'/Yellow'+ year + '_drop_'+ month + '.csv')
# del pick16['Unnamed: 0']
# del drop16['Unnamed: 0']

# df_pick = pd.merge(df_pick, pick16, how='outer', on = 'ZIPCODE')
# df_drop = pd.merge(df_drop, drop16, how='outer', on = 'ZIPCODE')

In [99]:
#year = Year[16] #define the year
#read the first month data
df_pick = pd.read_csv('Yellow_taxi_data/20'+ year +'/Yellow'+ year + '_pick_'+ Month[1] + '.csv')
df_drop = pd.read_csv('Yellow_taxi_data/20'+ year +'/Yellow'+ year + '_drop_'+ Month[1] + '.csv')
del df_pick['Unnamed: 0']
del df_drop['Unnamed: 0']

#merge the monthly data one by one
for i in range(2,13):
    next_pick = pd.read_csv('Yellow_taxi_data/20'+ year +'/Yellow'+ year + '_pick_'+ Month[i] + '.csv')
    next_drop = pd.read_csv('Yellow_taxi_data/20'+ year +'/Yellow'+ year + '_drop_'+ Month[i] + '.csv')
    del next_pick['Unnamed: 0']
    del next_drop['Unnamed: 0']
    df_pick = pd.merge(df_pick, next_pick,how='outer', on = 'ZIPCODE')
    df_drop = pd.merge(df_drop, next_drop,how='outer', on = 'ZIPCODE')

In [100]:
df_pick.to_csv('Yellow_taxi_data/20'+ year +'/Yellow' + year + '_pick_monthly1.csv')
df_drop.to_csv('Yellow_taxi_data/20'+ year +'/Yellow' + year + '_drop_monthly1.csv')

## Merge yearly data

In [ ]:
# #year = Year[16] #define the year
# df_pick = pd.read_csv('Yellow_taxi_data/20'+ Year[9] +'/Yellow'+ year + '_pick_monthly.csv')
# df_drop = pd.read_csv('Yellow_taxi_data/20'+ Year[9] +'/Yellow'+ year + '_pick_monthly.csv')
# del df_pick['Unnamed: 0']
# del df_drop['Unnamed: 0']

# pick16 = pd.read_csv('Yellow_taxi_data/20'+ Year[16] +'/Yellow' + Year[16] + '_pick_monthly1.csv')
# drop16 = pd.read_csv('Yellow_taxi_data/20'+ Year[16] +'/Yellow' + Year[16] + '_pick_monthly1.csv')
# del pick16['Unnamed: 0']
# del drop16['Unnamed: 0']

# #merge from 2009 to 2015
# for i in range(10,16): #merge 09-15
#     next_pick = pd.read_csv('Yellow_taxi_data/20'+ Year[i] +'/Yellow'+ year + '_pick_'+ Month[i] + '.csv')
#     next_drop = pd.read_csv('Yellow_taxi_data/20'+ Year[i] +'/Yellow'+ year + '_drop_'+ Month[i] + '.csv')
#     del next_pick['Unnamed: 0']
#     del next_drop['Unnamed: 0']
#     df_pick = pd.merge(df_pick, next_pick,how='outer', on = 'ZIPCODE')
#     df_drop = pd.merge(df_drop, next_drop,how='outer', on = 'ZIPCODE')

# #merge df_pick with 2016
# df_pick = pd.merge(df_pick, pick16, how='outer', on = 'ZIPCODE') 
# df_drop = pd.merge(df_drop, drop16, how='outer', on = 'ZIPCODE')

In [ ]:
# df_pick

In [ ]:
# df_drop

In [ ]:
# df_pick.to_csv('Yellow_taxi_data/Yellow_taxi' + year + '_pick_monthly(09-16).csv')
# df_drop.to_csv('Yellow_taxi_data/Yellow_taxi' + year + '_drop_monthly(09-16).csv')

In [163]:
# test1 = pd.read_csv('Yellow_taxi_data/Yellow17_pick_monthly.csv')
# test2 = pd.read_csv('Yellow_taxi_data/Yellow17_drop_monthly.csv')

In [164]:
# del test1['Unnamed: 0']
# del test2['Unnamed: 0']

In [165]:
# test1 = pd.merge(pick,test1,how='outer',on='LocationID')
# test2 = pd.merge(drop,test2,how='outer',on='LocationID')

In [166]:
# test1.to_csv('Yellow_taxi_data/Yellow_pick_monthly(16-17).csv')
# test2.to_csv('Yellow_taxi_data/Yellow_drop_monthly(16-17).csv')